# Some startup code

import packages needed & load dataset

In [1]:
from sklearn import svm, datasets, preprocessing
import numpy as np
from loaddata import get_instances_from_csv
from startEvaluation import evaluation

import sys
import threading
import time
from contextlib import contextmanager

DATAPATH = "../sub_datasets/subset_0.csv"

training_data = get_instances_from_csv(DATAPATH, "train", numrows=50000)
eval_data = get_instances_from_csv(DATAPATH,"eval", numrows=500000)
print("training data  : %s rows"%training_data[0].shape[0])
print("evaluation data: %s rows"%eval_data[0].shape[0])

kernels = ['rbf', 'linear', 'sigmoid', 'poly']

scaler = preprocessing.StandardScaler()
scaler.fit(training_data[0])

def print_prediction(training_data, eval_data, *args, **kwargs):

    print("start training")
    svc = svm.SVC(*args, **kwargs).fit(training_data[0], training_data[1])
    print("finished training svm %s"%svc)
    evt = evaluation(training_data[1], svc.predict(training_data[0]))
    print("accuracy on traning data:")
    evt.print_only_accuracy()
    eve = evaluation(eval_data[1], svc.predict(eval_data[0]))
    print("accuracy on evaluation data:")
    eve.print_only_accuracy()


/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


features:  [['-4.9667664' '-1.8327484' '10.064636' '-0.10971069' '0.048919678'
  '0.16738892']
 ['-5.9427185' '0.6761626999999999' '8.128204' '0.016952515' '-0.003829956'
  '-0.02017212']
 ['-4.6073303' '0.6856842' '7.863983' '-0.59783936' '0.43984985'
  '1.1532593']
 ..., 
 ['-3.371036' '-0.33518824' '3.9647980000000005' '-0.0076358155'
  '0.025350908' '-0.0012217305']
 ['-4.918991' '1.3883362' '9.174178999999999' '0.15565489999999998'
  '-0.55152893' '0.47473145']
 ['0.46183777' '-0.7813720999999999' '9.863296499999999'
  '-0.0035552978999999997' '-0.012924193999999998' '0.00041198730000000005']]
class_labels:  ['3' '2' '4' ..., '3' '0' '1']
features:  [[-5.4779334 -0.9959879 8.149861999999999 -1.3838496999999998 2.047961
  -0.48869026]
 [4.0462008 1.650802 8.422802 -0.0073394775 -0.009567261 0.027053833]
 [4.048595 1.6783354 8.444349 -0.0062713623 -0.008499145500000001
  0.029190063]
 ..., 
 [-6.2378845 0.29411316 9.556427 0.11271837 0.06685029 0.06026285]
 [-5.0560303 -0.056991577 

## Basic SVM
As you can see the print_prediction function fits the SVM-Model to the training data and prints a very simple evaluation.

In [2]:
print_prediction(training_data, eval_data)

start training
finished training svm SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.877277545551
accuracy on evaluation data:
0.539625436462



## Lets use different kernels and see how the perform
Currently there are 3 Kernels implemented by sklearn that we can test. (Of course we could also implement own kernels later on)

In [3]:
for kernel in kernels:
    print(kernel)
    print_prediction(training_data, eval_data, kernel=kernel, cache_size=1000)

rbf
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.656450463293
accuracy on evaluation data:
0.539625436462
linear
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.553215434756
accuracy on evaluation data:
0.522657214404
sigmoid
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on tranin

The RBF kernel seems to perform slightly better then the linear kernel. Both are far better then the sigmoid-kernel.

## Lets also check, if scaling the data improves the classification performance
Luckily sklearn also provides features for scaling data, the standart scaling function from sklearn scales the data in a way that variance == 1 and mean == 0.

In [4]:
training_data_scaled = [scaler.transform(training_data[0]), training_data[1]]
eval_data_scaled = [scaler.transform(eval_data[0]), eval_data[1]]
for kernel in kernels:
    print(kernel)
    print_prediction(training_data_scaled, eval_data_scaled, kernel=kernel, cache_size=1000)

rbf
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.499195166796
accuracy on evaluation data:
0.482528832928
linear
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.494204371265
accuracy on evaluation data:
0.483530843297
sigmoid
start training
finished training svm SVC(C=1.0, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
accuracy on tranin

## Parameter Variations

In [ ]:

for gamma in range(.1, .99, .1):
    for kernel in kernels:
        print(kernel)
        print_prediction(training_data, eval_data, kernel=kernel, cache_size=1000)
        
for coef0 in range(.1, 2.0, .2):
    for kernel in ['poly', 'sigmoid']:
        print(kernel)
        print_prediction(training_data, eval_data, kernel=kernel, cache_size=1000)
        
for decision_function_shape in ['ovo', 'ovr']:
    for kernel in kernels:
        print(kernel)
        print_prediction(training_data, eval_data, kernel=kernel, cache_size=1000)
        
for degree in range(1, 5):
    for kernel in ['poly']:
        print(kernel)
        print_prediction(training_data, eval_data, kernel=kernel, cache_size=1000)
